# POP MOC(sigma 2) for 0.1-degree  -- SUNWAY cases
**Input Data:** Monthly POP output timeseries files  
**Output Data:** Monthly mean AMOC sigma 2 timeseries  
**Description:** Computes MOC(sigma 2) offline from POP history files using simple xhistogram binning.  
**Date:** February 2023  
**Creator:** Steve Yeager (https://github.com/sgyeager/POP_MOC/blob/main/notebooks/pop_MOCsigma2_0.1deg.ipynb)  
**Updated:** Teagan King, February 2023  
**Note:** To use the MOCutils, a user will need to clone the POP_MOC repository (https://github.com/sgyeager/POP_MOC) and install MOCutils by going to the POP_MOC directory and running `pip install -e . --user`.  
It is also important to request enough memory for this notebook on a casper batch node; suggested allocation is 25GB.

In [2]:
%load_ext autoreload
%autoreload 2
import cftime
import copy
import dask
import glob
import matplotlib.pyplot as plt
%matplotlib inline
from MOCutils import popmoc
import numpy as np  
import os
import pop_tools
import time
import xarray as xr 
from xhistogram.xarray import histogram

In [3]:
from dask.distributed import wait
dask.__version__

'2022.11.0'

In [4]:
pop_tools.__version__

'2023.3.0'

In [5]:
# Close out Dask Cluster and release workers:
# cluster.close()
# client.close()

In [6]:
def get_ClusterClient():
    import dask
    from dask_jobqueue import PBSCluster
    from dask.distributed import Client
    cluster = PBSCluster(
        cores=1,
        memory='20GiB',
        processes=1,
        queue='casper',
        resource_spec='select=1:ncpus=1:mem=20GB', 
        account='P93300313',
        walltime='06:00:00',
        interface='ib0',)

    dask.config.set({
        'distributed.dashboard.link':
        'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status',
        "distributed.scheduler.worker-saturation": "1.0",
        'array.slicing.split_large_chunks': True
    })
    client = Client(cluster)
    return cluster, client

cluster, client = get_ClusterClient()
cluster.scale(50) 

/glade/u/home/tking/.conda/envs/ipogs_from_steve/lib/python3.10/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39317 instead
  warnings.warn(


In [7]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/tking/proxy/39317/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/tking/proxy/39317/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.54:32879,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/tking/proxy/39317/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [8]:
def time_set_midmonth(ds, time_name, deep=False):
    """
    Return copy of ds with values of ds[time_name] replaced with mid-month
    values (day=15) rather than end-month values.
    """
    year = ds[time_name].dt.year
    month = ds[time_name].dt.month
    year = xr.where(month==1,year-1,year)
    month = xr.where(month==1,12,month-1)
    nmonths = len(month)
    newtime = [cftime.DatetimeNoLeap(year[i], month[i], 15) for i in range(nmonths)]
    ds[time_name] = newtime
    return ds

# Get the required variables 

In [ ]:
fdir = '/glade/campaign/collections/cmip/CMIP6/iHESP/BRCP85/HR/B.E.13.BRCP85C5CN.ne120_t12.sehires38.003.sunway.CN_OFF/ocn/proc/tseries/month_1/'
fin = fdir + 'B.E.13.BRCP85C5CN.ne120_t12.sehires38.003.sunway.CN_OFF.pop.h.'
vvel_files = sorted(glob.glob(fin+'VVEL*.nc'))
wvel_files = sorted(glob.glob(fin+'WVEL*.nc'))
temp_files = sorted(glob.glob(fin+'TEMP*.nc'))
salt_files = sorted(glob.glob(fin+'SALT*.nc'))
uvel_files = sorted(glob.glob(fin+'UVEL*.nc'))

fgrd = '/glade/work/fredc/cesm/grid/POP/grid.3600x2400x62.nc'
ds_grid = xr.open_dataset(fgrd)

fmoc = '/glade/u/home/yeager/analysis/python/POP_MOC/moc_template.nc'
ds_moctemp = xr.open_dataset(fmoc)

## Define the MOC region mask:
rmask = ds_grid.REGION_MASK.drop(['ULONG','ULAT'])
rmaskglob = xr.where((rmask>0),1,0)
rmaskatl = xr.where((rmask>=6) & (rmask<=11),1,0)
rmaskmoc = xr.concat([rmaskglob,rmaskatl],dim=ds_moctemp.transport_regions)

# determine j=index of Atlantic region southern boundary
tmp = rmaskmoc.isel(transport_reg=1).sum('nlon')
atl_j = 0
j = 0
while (atl_j==0):
    if (tmp.isel(nlat=j).data>0):
        atl_j = j
    j += 1
atl_j = atl_j - 1

dz = ds_grid['dz'].persist() / 100.
kmt = ds_grid['KMT'].fillna(0).persist() 
print('got dz and kmt')
# Slow step (~12 mins)
dzt,dzu = popmoc.tx0p1v3_dztdzu(dz,kmt)
print('got dzt dzu')

# Compute sigma-2 field from POP model output
refz = 2000
refdep = xr.DataArray(refz)

# Grid Metrics
dxu = ds_grid['DXU']
dyu = ds_grid['DYU']
dxt = ds_grid['DXT']
dyt = ds_grid['DYT']

In [20]:
for n in range(len(vvel_files)):
    dsV = xr.open_mfdataset(vvel_files[n], chunks={'time':1,'nlat':100}, parallel=True)
    dsV = time_set_midmonth(dsV,'time')
    dsW = xr.open_mfdataset(wvel_files[n], chunks={'time':1,'nlat':100}, parallel=True)
    dsW = time_set_midmonth(dsW,'time')
    dsT = xr.open_mfdataset(temp_files[n], chunks={'time':1,'nlat':100}, parallel=True)
    dsT = time_set_midmonth(dsT,'time')
    dsS = xr.open_mfdataset(salt_files[n], chunks={'time':1,'nlat':100}, parallel=True)
    dsS = time_set_midmonth(dsS,'time')
    dsU = xr.open_mfdataset(uvel_files[n], chunks={'time':1,'nlat':100}, parallel=True)
    dsU = time_set_midmonth(dsU,'time')

    u_e_all = dsU['UVEL']
    u_e_all = u_e_all.where(u_e_all<1.e30,0)
    v_e_all = dsV['VVEL']
    v_e_all = v_e_all.where(v_e_all<1.e30,0)

    # Get model T & S
    salt_all = dsS['SALT']
    temp_all = dsT['TEMP']

    tlon = ds_grid.TLONG.drop(['ULONG','ULAT'])
    tlat = ds_grid.TLAT.drop(['ULONG','ULAT'])
    ulon = ds_grid.ULONG.drop(['TLONG','TLAT'])
    ulat = ds_grid.ULAT.drop(['TLONG','TLAT'])

    print("started MOC calculations {}".format(time.ctime()))
    v_e = v_e_all
    u_e = u_e_all
    salt = salt_all
    temp = temp_all

    # Sigma2 on model TLAT, TLONG
    sigma2_T = pop_tools.eos(salt=salt,temp=temp,depth=refdep) - 1000
    sigma2_T = sigma2_T.assign_attrs({'long_name':'Sigma referenced to {}m'.format(refz),'units':'kg/m^3'})

    # Define target sigma-2 vertical grid. Use a predefined target grid, or create your own!
    sigma_mid,sigma_edge = popmoc.sigma2_grid_86L()

    # Grid-oriented Volume FLuxes:
    u_e = (u_e*dyu*dzu/1.e4).assign_attrs({'units':'m^3/s'})
    v_e = (v_e*dxu*dzu/1.e4).assign_attrs({'units':'m^3/s'})

    # Convert u_e,v_e to C-grid fluxes
    u = 0.5*(u_e+u_e.shift(nlat=1))
    v = 0.5*(v_e+v_e.roll(nlon=1,roll_coords=False))

    # Volume fluxes in density-space. 
    iso_uflux = histogram(sigma2_T, bins=[sigma_edge.values],weights=u,dim=['z_t'],density=False)
    iso_uflux = iso_uflux.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})
    iso_vflux = histogram(sigma2_T, bins=[sigma_edge.values],weights=v,dim=['z_t'],density=False)
    iso_vflux = iso_vflux.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})

    # Compute Vertical Volume Flux from horizontal flux convergence
    wflux = popmoc.wflux(iso_uflux,iso_vflux,'sigma',sigma_edge,grid='C')
    wflux = wflux.assign_coords({'TLAT':tlat,'TLONG':tlon})

    # Compute MOC
    MOC = popmoc.compute_MOC(wflux,rmaskmoc,ds_moctemp.lat_aux_grid)
    MOC = MOC.transpose('time','transport_reg','sigma','lat_aux_grid')
    MOC = dask.optimize(MOC)[0]

    # add vflux at southern boundary of Atlantic domain
    tmp = iso_vflux*(rmaskmoc.shift(nlat=-1))
    tmp = tmp.isel(nlat=atl_j,transport_reg=1).sum('nlon')
    moc_s = -tmp.sortby('sigma',ascending=False).cumsum('sigma').sortby('sigma',ascending=True)/1.e6
    moc_s['sigma'] = sigma_edge.isel(sigma=slice(0,-1))
    MOC[{'transport_reg':1}] = MOC[{'transport_reg':1}] + moc_s

    # Save to netcdf
    MOCann = MOC.groupby('time.year').mean('time').rename({'year':'time'})
    dsout = MOCann.to_dataset()

    outdir = '/glade/scratch/tking/MOCsig/'
    fout = os.path.split(vvel_files[n])[-1].split('.')[:-3]
    fout.append('MOCsig')
    fout.append('{:04d}01-{:04d}12'.format(dsout.time.values[0],dsout.time.values[-1]))
    fout.append('nc')
    fout = '.'.join(fout)
    fout = os.path.join(outdir,fout)
    dsout.to_netcdf(fout,unlimited_dims='time')
    print("wrote {} at {}".format(fout, time.ctime()))

got dz and kmt
got dzt dzu
started MOC calculations Mon Mar 13 11:57:27 2023
wrote /glade/scratch/tking/MOCsig/B.E.13.BRCP85C5CN.ne120_t12.sehires38.003.sunway.CN_OFF.pop.h.MOCsig.200601-200612.nc at Mon Mar 13 11:59:59 2023
started MOC calculations Mon Mar 13 12:00:09 2023
wrote /glade/scratch/tking/MOCsig/B.E.13.BRCP85C5CN.ne120_t12.sehires38.003.sunway.CN_OFF.pop.h.MOCsig.200701-200712.nc at Mon Mar 13 12:02:48 2023
started MOC calculations Mon Mar 13 12:02:52 2023
wrote /glade/scratch/tking/MOCsig/B.E.13.BRCP85C5CN.ne120_t12.sehires38.003.sunway.CN_OFF.pop.h.MOCsig.200801-200812.nc at Mon Mar 13 12:05:19 2023
started MOC calculations Mon Mar 13 12:05:22 2023
wrote /glade/scratch/tking/MOCsig/B.E.13.BRCP85C5CN.ne120_t12.sehires38.003.sunway.CN_OFF.pop.h.MOCsig.200901-200912.nc at Mon Mar 13 12:07:50 2023
started MOC calculations Mon Mar 13 12:07:53 2023
wrote /glade/scratch/tking/MOCsig/B.E.13.BRCP85C5CN.ne120_t12.sehires38.003.sunway.CN_OFF.pop.h.MOCsig.201001-201012.nc at Mon Mar 